In [37]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from os.path import exists
from pydub import AudioSegment
from TTS.api import TTS
import os

import sys
sys.path.insert(0, "../Src/")

%autosave 5

Autosaving every 5 seconds


In [38]:
#dataSet = pd.read_csv('../Data/train_data.csv')
#dataSet[dataSet['speaker_id'] == 'MMDM0']

In [39]:
def readCsv(dataset):
    return pd.read_csv(f'../Data/{dataset}_data.csv')

In [40]:
def getSpeakers(df):
    speakerIds = df['speaker_id']
    return list(set(speakerIds))

In [41]:
def getFilesBySpeaker(df,speakerId):
    return df[df['speaker_id']==speakerId]

In [42]:
def concatenateAudio(speakerId, speakerDF):
    if speakerDF.empty:
        print(f"Empty DataFrame for speaker {speakerId}. Skipping.")
        return

    finalAudioFile = f'../Data/concatenatedInputs/{speakerId}.wav'
    audioData = speakerDF[speakerDF['filename'].str.endswith('.wav', na=False)]

    if audioData.empty:
        print(f"No .wav files for speaker {speakerId}. Skipping.")
        return

    audioFileList = audioData['path_from_data_dir']

    if not exists(finalAudioFile):
        concat_audio = AudioSegment.empty()
    else:
        concat_audio = AudioSegment.from_wav(finalAudioFile)
    for audioFile in audioFileList:
        try:
            audio = AudioSegment.from_wav(f'../Data/data/{audioFile}') + AudioSegment.silent(duration=1000)
            concat_audio += audio
        except Exception as e:
            print(f"Failed to load {audioFile}: {e}")

    if len(concat_audio) > 0:
        concat_audio.export(finalAudioFile, format='wav')
    else:
        print(f"No valid audio for speaker {speakerId}, nothing exported.")
    

In [43]:
def generateAudio(speakers, trainDF):
    tts_instance = TTS()  # Create an instance of the TTS class
    englishModels = [model for model in tts_instance.list_models() if 'en' in model]

    for model in englishModels:
        modelDirectory = f'../Data/ttsOutputs/{model}'
        if not exists(modelDirectory):
            os.makedirs(modelDirectory)
        tts = TTS(model_name=model, progress_bar=False, gpu=True)
        for speaker in speakers:
            speakerDF = getFilesBySpeaker(trainDF, speaker)
            if not speakerDF.empty:
                audioFile = f'../Data/concatenatedInputs/{speaker}.wav'
                if exists(audioFile):
                    tts.tts_to_file(
                        text=speakerDF.iloc[0]['text'],
                        file_path=f'{modelDirectory}/{speaker}.wav',
                        speaker_id=speaker
                        )
                else:
                    print(f"Audio file for {speaker} does not exist.")

In [44]:
def main():
    trainDF = readCsv('train')
    speakers = getSpeakers(trainDF)
    if not exists('../Data/concatenatedInputs'):
        os.makedirs('../Data/concatenatedInputs')
        for speaker in speakers:
            speakerDF = getFilesBySpeaker(trainDF,speaker)
            concatenateAudio(speaker,speakerDF)
    if not exists('../Data/ttsOutputs/'):
        os.makedirs('../Data/ttsOutputs/')
    generateAudio(speakers, trainDF)
    
    

In [45]:
if __name__ == '__main__':
    main()

TypeError: 'ModelManager' object is not iterable